In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


In [2]:

# Load your dataset
# Assuming you have a .npy file for data (adjust the path as necessary)
x = np.load("/Users/rohitviswam/Downloads/CSI-02-0005/H_32T4R_30_1RB.npy").reshape(2100, 398, 256, 1)  # Add a channel dimension
# Create synthetic labels for demonstration (adjust according to your actual labels or tasks)
y = np.arange(2100) % 10  # Example: 10 classes



In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [4]:

class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule

    def build(self, input_shape):
        self.W = self.add_weight(shape=[input_shape[-1], self.num_capsule * self.dim_capsule],
                                 initializer='glorot_uniform',
                                 trainable=True)

    def call(self, inputs):
        u = tf.expand_dims(inputs, -2)
        u_hat = tf.reshape(tf.matmul(u, self.W), (-1, inputs.shape[1], self.num_capsule, self.dim_capsule))
        v = tf.reduce_sum(u_hat, axis=2)  # Sum over inputs, only keep capsules
        return v


In [5]:
def create_capsnet_model():
    inputs = layers.Input(shape=(398, 256, 1))
    x = layers.Conv2D(32, (9, 9), strides=(2, 2), activation='relu', padding='valid')(inputs)
    x = layers.Dropout(0.25)(x)  # Adding dropout
    x = layers.Reshape((-1, 32))(x)
    x = CapsuleLayer(10, 16)(x)
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust learning rate
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [6]:

# Create the model
model = create_capsnet_model()
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 398, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 195, 124, 32)   │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 195, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 24180, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capsule_layer (CapsuleLayer)    │ (None, 24180, 16)      │         5,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │           170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,914 (30.91 KB)

 Trainable params: 7,914 (30.91 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:

# Train the model
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), batch_size=8)


Epoch 1/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 621s 3s/step - accuracy: 0.0945 - loss: 2.3194 - val_accuracy: 0.1048 - val_loss: 2.3043
Epoch 2/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 415s 2s/step - accuracy: 0.1210 - loss: 2.2960 - val_accuracy: 0.1119 - val_loss: 2.3246
Epoch 3/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 292s 1s/step - accuracy: 0.1193 - loss: 2.2906 - val_accuracy: 0.0786 - val_loss: 2.3194
Epoch 4/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 183s 867ms/step - accuracy: 0.1450 - loss: 2.2777 - val_accuracy: 0.1000 - val_loss: 2.3279
Epoch 5/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 279s 1s/step - accuracy: 0.1279 - loss: 2.2741 - val_accuracy: 0.0929 - val_loss: 2.3435
Epoch 6/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 520s 2s/step - accuracy: 0.1414 - loss: 2.2653 - val_accuracy: 0.0929 - val_loss: 2.3288
Epoch 7/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 6230s 30s/step - accuracy: 0.1575 - loss: 2.2535 - val_accuracy: 0.0857 - val_loss: 2.3429
Epoch 8/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 182s 862ms/step - accuracy: 0.1546 - loss: 2

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_acc)


14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 124ms/step - accuracy: 0.1114 - loss: 2.3407
Test Accuracy: 0.1071428582072258


In [ ]:
# Optionally, save the model for later use
model.save('my_capsnet_model2.h5')
